通过高德地图开放平台爬取延安东路立交桥区域路段的交通状态

In [2]:
import requests
import json
import csv
import datetime
import schedule
import math
import threading
import queue
from fake_useragent import UserAgent

In [58]:
q = queue.Queue()
thread_list = []

In [59]:
def get_data(coord1,coord2):
    server_host='https://restapi.amap.com/v3/traffic/status/rectangle'
    parameters={
        'key':'111d1e3797ffa20c7d44a42ab0bed3fd',
        'level':5,
        'extensions':'all',
        'rectangle':f'{coord1[0]},{coord1[1]};{coord2[0]},{coord2[1]}'
    }
    response=requests.get(url=server_host,
                         headers={'User-Agent':UserAgent(path='fakeuseragent.json').random},
                         params=parameters)
    response.encoding=response.apparent_encoding
    return response.text

csv文件各列属性依次为：name, status, direction, angle, speed, lcodes, polyline

In [60]:
def save_as_csv(time,data):
    with open(f'data/ts_{time}.csv','w') as f:
        csv_writer = csv.writer(f)
        for item in data:
            if 'speed' in item:
                csv_writer.writerow([item['name'],
                                     item['status'],
                                     item['direction'],
                                     item['angle'],
                                     item['speed'],
                                     item['lcodes']
                                    ])
        f.close()

In [61]:
def download_task(coord1,coord2):
    #
    res=json.loads(get_data(coord1,coord2))
    data=[road for road in res['trafficinfo']['roads']]
    q.put(data)

设置schedule定时任务

In [62]:
def run():
    time=(datetime.datetime.now()+datetime.timedelta(hours=8)).strftime('%Y-%m-%d %H:%M:%S')
    size=6#(size*5)^2 km^2
    coord0=(121.475698,31.230428)
    coord1=(coord0[0]-5*float(size/2)/(111.0*math.cos(coord0[1])),coord0[1]-5*float(size/2)/111.0)
    coord2=(coord0[0]+5*float(size/2)/(111.0*math.cos(coord0[1])),coord0[1]+5*float(size/2)/111.0)
    for i,lng in enumerate([coord1[0]+i*(coord2[0]-coord1[0])/size for i in range(size)]):
        for j,lat in enumerate([coord1[1]+j*(coord2[1]-coord1[1])/size for j in range(size)]):
            args=((lng,lat),(lng+(coord2[0]-coord1[0])/size,lat+(coord2[1]-coord1[1])/size))
            thread = threading.Thread(target=download_task, args=args)
            thread_list.append(thread)
            thread.start()
    for thread in thread_list:
        thread.join()
    thread_list.clear()
    data=[]
    while not q.empty():
        data.extend(q.get())
    save_as_csv(time,data)

In [8]:
schedule.every(5).minutes.do(run)
while True:
    start_time=datetime.datetime.strptime(str((datetime.datetime.now()+datetime.timedelta(hours=8)).date())+' 5:00:00','%Y-%m-%d %H:%M:%S')
    end_time=datetime.datetime.strptime(str((datetime.datetime.now()+datetime.timedelta(hours=8)).date())+' 23:59:59','%Y-%m-%d %H:%M:%S')
    now_time = datetime.datetime.now()+datetime.timedelta(hours=8)
    print(now_time)
    if start_time < now_time < end_time:
        schedule.run_pending()

2021-10-01 09:31:23.548238
1
